In [1]:
import pandas as pd
import numpy as np
import torch
import csv
from sentence_transformers import SentenceTransformer, util
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as func
from torch.nn import Linear as lin
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import string
from nltk import word_tokenize
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import spacy
from torch.distributions.multinomial import Multinomial
from transformers import BertForTokenClassification
from transformers import BertForSequenceClassification
from transformers import AutoTokenizer
from transformers import DistilBertTokenizerFast
from transformers import BertTokenizer
from transformers import DistilBertForTokenClassification
from transformers import BertForMaskedLM
from transformers import BertTokenizerFast

/home/henry/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-10 13:01:16.780257: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 13:01:16.780799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 13:01:16.780936: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
df = pd.read_csv("Train_Tagged_Titles.tsv.gz", sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)

In [3]:

for i in range(len(df)):
    if type(df["Tag"][i]) == float:
        df["Tag"][i] = "Next"
    

In [4]:
vec_dict = {}
with open("glove.840B.300d.txt", 'r', encoding="utf-8") as f:
    for line in f:
        vals = line.split(" ")
        vec_dict[vals[0]] = np.asarray(vals[1:], dtype="float32")

In [5]:
model = SentenceTransformer('all-mpnet-base-v2', device="cuda")
rows = []
duds = {}
is_mpn = []
for i in set(df["Tag"].values.tolist()):
    duds[i] = []
nt = []
cnt = 0
for i in range(1,5001):
    if i%100 == 0:
        print(i)
    listing = df[df["Record Number"] == str(i)]
    transformed = []
    #sent_vec = model.encode(listing["Title"].values[0].lower())
    for j in range(1,len(listing)):
        prev = listing.iloc[j-1]
        row = listing.iloc[j]
        if prev["Token"].lower() in vec_dict and row["Token"].lower() in vec_dict:
            prev_vec = vec_dict[prev["Token"].lower()]
            tok_vec = vec_dict[row["Token"].lower()]
            if row["Tag"] == "Next":
                enc = np.array([1])
            else:
                enc = np.array([0])
            transformed.append(np.hstack((prev_vec, tok_vec, enc)))
        else:
            v = np.zeros((300,))
            div = len(row["Token"])
            for let in row["Token"]:
                if let in vec_dict:
                    v = v + vec_dict[let]
                else:
                    div -= 1
            if div != 0:
                tok_vec = v/div
            else:
                tok_vec = vec_dict["number"]
            b = np.zeros((300,))
            div = len(prev["Token"])
            for let in prev["Token"]:
                if let in vec_dict:
                    b = b + vec_dict[let]
                else:
                    div -= 1
            if div != 0:
                prev_vec = b/div
            else:
                prev_vec = vec_dict["number"]

            if row["Tag"] == "Next":
                enc = np.array([1])
            else:
                enc = np.array([0])

            transformed.append(np.hstack((prev_vec, tok_vec, enc)))
        
    rows += transformed

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [6]:
def get_text(df):
    text = []
    labels = []
    for i in range(1,5001):
        listing = df[df["Record Number"] == str(i)]
        
        text.append(listing["Token"].values.tolist())
        labels.append(listing["Tag"].values.tolist())
    return text, labels

In [7]:
tag_to_id = {}
cnt = 0
for i in set(df["Tag"].values.tolist()):
    
    tag_to_id[i] = cnt
    cnt += 1

In [8]:
id_to_tag = {}
cnt = 0
for i in set(df["Tag"].values.tolist()):
    id_to_tag[cnt] = i
    cnt += 1

In [9]:
text, labels = get_text(df)

In [10]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
all_encodings = tokenizer(text, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [11]:
def encode_tags(tags, encodings):
    labels = []
    for doc in tags:
        f = []
        for tag in doc:
            f.append(tag_to_id[tag])
        labels.append(f)
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        d = doc_offset
        for i in range(40 - len(d)):
            d.append((0,0))
        doc_enc_labels = np.ones(40,dtype=int) * -100
        arr_offset = np.array(d)
        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = np.array(doc_labels)
        """last = 0
        c = arr_offset[:,1].flatten()
        for v in range(1, len(c)):
            if c[v] == 0:
                for lab in range(1, v+1):
                    if doc_enc_labels[lab] != -100:
                        last = doc_enc_labels[lab]
                    if doc_enc_labels[lab] == -100:
                        doc_enc_labels[lab] = last
                break
            """

        encoded_labels.append(doc_enc_labels.tolist())
    

    return encoded_labels

train_tags = encode_tags(labels, all_encodings)

In [12]:
bert = BertForMaskedLM.from_pretrained('bert-base-uncased', num_labels=33, id2label=id_to_tag).to("cpu")
r = []
for i in range(250,5001,250):
    print(i)
    r+= bert.bert(input_ids=torch.tensor(all_encodings["input_ids"][i-250:i]).cpu(), attention_mask=torch.tensor(all_encodings["attention_mask"][i-250:i]).cpu())[0].cpu().tolist()
r = torch.tensor(r)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
3000
3250
3500
3750
4000
4250
4500
4750
5000


In [13]:
label_idx = np.where(np.array(train_tags) != -100)
bert_out = r.numpy()[label_idx]

In [20]:
len(bert_out[0])

768

In [14]:
bert_data = np.zeros((1537,))
count = 0
for m in range(1,5001):
    listing = df[df["Record Number"] == str(m)]
    bert_listing = bert_out[count:count + len(listing), :]
    bert_listing = np.hstack((bert_listing[:len(bert_listing)-1, :], bert_listing[1:,:]))
    is_next = []
    for j in range(1, len(listing)):
        prev = listing.iloc[j]
        if prev["Tag"] == "Next":
            is_next.append(1)
        else:
            is_next.append(0)
    count += len(listing)
    bert_data = np.vstack((bert_data, np.hstack((bert_listing, np.array(is_next).reshape(len(is_next), 1)))))
bert_data = bert_data[1:,:]

In [15]:
def train_loop_next(model, sample_weights=np.array([]), learning_rate=0.0001, l1=0.00001, batch_size=250, max_iter=3000, sample_func=1/2, train=None, test=None):
    train_s = train
    test_s = test
    loss_fn = torch.nn.BCELoss(reduction="mean")
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    current_min = 999999999
    num_since = 0

    best_params = model.parameters()
    max_val_f1 = 0
    max_train_f1 = 0
    for t in range(200000):
        perm = torch.randperm(len(train_s))
        x_full = train_s[perm][:250]
        x_train = x_full[:,:-1].cuda()
        y_train = x_full[:,-1].cuda()
        y_pred = model(x_train)
        loss = loss_fn(y_pred.flatten(), y_train)
        for i in model.parameters():
            loss += (torch.square(i).sum())*l1
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step(lambda: loss)
        if t%2000 == 0:
            temp_test = test_s[:,-1].flatten()
            temp_train = train_s[:,-1].flatten()
            t1 = test_s[:,:-1]
            t2 = train_s[:,:-1]
            test_out = model(t1).flatten()
            train_out = model(t2).flatten()
            val_score = 0
            train_score = 0
            for n in range(len(test_out)):
                if test_out[n] >= 0.5 and temp_test[n] == 1:
                    val_score += 1
                elif test_out[n] <= 0.5 and temp_test[n] == 0:
                    val_score += 1
            for n in range(len(train_out)):
                if train_out[n] >= 0.5 and temp_train[n] == 1:
                    train_score += 1
                elif train_out[n] <= 0.5 and temp_train[n] == 0:
                    train_score += 1
                
            print(t, loss)
            print("train f1: ", train_score/len(train_out))
            print("val f1: ",val_score/len(test_out))
            if val_score > max_val_f1:
                max_val_f1 = val_score
                best_params = model.parameters()
            if train_score > max_train_f1:
                max_train_f1 = val_score
                best_params = model.parameters()
        if loss.item() < current_min:
            num_since = 0
            current_min = loss
            
        elif num_since >= max_iter:
            print("max iter")
            break
        
        else:
            num_since += 1
    
    return (val_score, train_score)


In [16]:
class Model_next(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(len(train_s[0])-1, 1024).cuda()
        self.l2 = nn.Linear(1024, 128).cuda()
        self.l3 = nn.Linear(128, 64).cuda()
        self.l7 = nn.Linear(64, 1).cuda()
        self.layers = [self.l1, self.l2, self.l3,self.l7]
    def forward(self, x):
        #d = nn.Dropout(p=0.8)
        x = self.l1(x)
        x = func.silu(x)
        #x = d(x)
        #skip = x
        x = self.l2(x)
        x = func.silu(x)
        #x = x + skip
        #skip = x
        x = self.l3(x)
        x = func.silu(x)
        #x = d(x)
        #x = x + skip
        #x = self.l4(x)
        #x = func.silu(x)
        x = self.l7(x)
        return torch.sigmoid(x)
    def parameters(self):
        params = []
        for p in self.layers:
            params.append(p.weight)
            params.append(p.bias)
        return params

In [30]:
#bert_data = np.hstack((bert_out, np.array(y).reshape(len(y), 1)))
#s = StandardScaler()
train_next_bert = bert_data[:,:-1]
train_next_bert = np.hstack((train_next_bert, bert_data[:,-1].reshape(len(bert_data[:,-1]),1)))

In [31]:
rows = np.array(rows)
#s = StandardScaler()
train_next = rows[:,:-1]
train_next = np.hstack((train_next,rows[:,-1].reshape(len(rows[:,-1]),1)))

In [19]:
p = PCA(n_components=50)
t = p.fit_transform(train_next[:,300:-1])
t2 = p.fit_transform(train_next[:,:300])
train_next = np.hstack((t,t2, train_next[:,-1].reshape(len(rows[:,-1]),1)))

In [20]:
p = PCA(n_components=100)
t = p.fit_transform(train_next_bert[:,768:-1])
t2 = p.fit_transform(train_next_bert[:,:768])
train_next_bert = np.hstack((t, t2, train_next_bert[:,-1].reshape(len(train_next_bert), 1)))

In [ ]:
nexts = train_next[np.where(rows[:,-1] == 1)]
not_nexts = train_next[np.where(rows[:,-1] == 0)]

In [ ]:
nexts_bert = train_next_bert[np.where(train_next_bert[:,-1] == 1)]
not_nexts_bert = train_next_bert[np.where(train_next_bert[:,-1] == 0)]

In [ ]:
nexts = np.hstack((nexts_bert[:,:-1], nexts))
not_nexts = np.hstack((not_nexts_bert[:,:-1], not_nexts))

In [ ]:
train_next = np.vstack((nexts, not_nexts[:10000,:]))

In [ ]:
train_s, test_s = train_test_split(train_pca, train_size=0.9)
train_s = torch.tensor(train_s).cuda().float()
test_s = torch.tensor(test_s).cuda().float()

In [32]:
train_next = np.hstack((train_next_bert[:,:-1], train_next))

In [34]:
train_s, test_s = train_test_split(train_next, train_size=0.9)
train_s = torch.tensor(train_s).cuda().float()
test_s = torch.tensor(test_s).cuda().float()

In [40]:
train_s, test_s = train_test_split(train_next_bert, train_size=0.8)
train_s = torch.tensor(train_s).cuda().float()
test_s = torch.tensor(test_s).cuda().float()

In [27]:
train_s, test_s = train_test_split(bert_data, train_size=0.9)
train_s = torch.tensor(train_s).cuda().float()
test_s = torch.tensor(test_s).cuda().float()

In [35]:
next_mdl = Model_next()

In [36]:
train_loop_next(next_mdl, learning_rate=0.00001, l1=0.000001, batch_size=250, max_iter=5000, train=train_s, test=test_s)

0 tensor(0.7544, device='cuda:0', grad_fn=<AddBackward0>)
train f1:  0.13848984659040525
val f1:  0.13445042888489928
2000 tensor(0.1910, device='cuda:0', grad_fn=<AddBackward0>)
train f1:  0.9391903875144099
val f1:  0.9353680430879713
4000 tensor(0.1445, device='cuda:0', grad_fn=<AddBackward0>)
train f1:  0.9504079099051166
val f1:  0.9401555954518253
6000 tensor(0.1315, device='cuda:0', grad_fn=<AddBackward0>)
train f1:  0.9566152345481954
val f1:  0.9429483343307401
8000 tensor(0.0899, device='cuda:0', grad_fn=<AddBackward0>)
train f1:  0.9613594040968343
val f1:  0.9449431478156792
10000 tensor(0.0764, device='cuda:0', grad_fn=<AddBackward0>)
train f1:  0.9673450385740888
val f1:  0.945741073209655
12000 tensor(0.0874, device='cuda:0', grad_fn=<AddBackward0>)
train f1:  0.9712911235257604
val f1:  0.9493317374825454
14000 tensor(0.0690, device='cuda:0', grad_fn=<AddBackward0>)
train f1:  0.9753923915935089
val f1:  0.9489327747855576
16000 tensor(0.0334, device='cuda:0', grad_fn=<

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_10643/867846342.py:1 in <cell line: 1>                                     │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_10643/867846342.py'                  │
│ /tmp/ipykernel_10643/1932548708.py:17 in train_loop_next                                  │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_10643/1932548708.py'                 │
│                                                                                           │
│ /home/henry/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1130 in        │
│ _call_impl                                                                                │
│                                                                                           │
│   1127 │   │   # this function, and just call forward.                                    │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1131 │   │   # Do not call functions when jit is used                                   │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /tmp/ipykernel_10643/1898263845.py:11 in forward                                          │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_10643/1898263845.py'                 │
│                                                                                           │
│ /home/henry/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1130 in        │
│ _call_impl                                                                                │
│                                                                                           │
│   1127 │   │   # this function, and just call forward.                                    │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1131 │   │   # Do not call functions when jit is used                                   │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /home/henry/.local/lib/python3.10/site-packages/torch/nn/modules/linear.py:114 in forward │
│                                                                                           │
│   111 │   │   │   init.uniform_(self.bias, -bound, bound)                                 │
│   112 │                                                                                   │
│   113 │   def forward(self, input: Tensor) -> Tensor:                                     │
│ ❱ 114 │   │   return F.linear(input, self.weight, self.bias)                              │
│   115 │                                                                                   │
│   116 │   def extra_repr(self) -> str:            

In [40]:
torch.save(next_mdl.state_dict(), "/home/henry/work/projects/next_params.obj")

In [41]:
model = Model_next()
model.load_state_dict(torch.load("/home/henry/work/projects/next_params.obj"))

<All keys matched successfully>

In [42]:
model.parameters()

[Parameter containing:
 tensor([[-7.8067e-03,  8.9061e-03, -2.3258e-03,  ..., -1.8071e-02,
           1.5533e-03, -8.7198e-03],
         [-1.3993e-02, -2.8622e-03, -1.0201e-02,  ...,  1.9151e-02,
          -7.5897e-03, -1.4495e-02],
         [-1.5968e-02,  1.7046e-02,  8.2233e-03,  ...,  1.4641e-03,
           7.0825e-04, -7.1359e-03],
         ...,
         [-1.5542e-03, -2.2315e-02,  3.7406e-03,  ...,  2.3636e-03,
           1.7146e-03, -4.2791e-03],
         [ 1.0300e-02, -1.8549e-02, -3.7147e-03,  ..., -1.2892e-02,
          -1.4822e-02, -1.0586e-02],
         [ 1.6928e-05, -1.2673e-02, -4.8640e-03,  ..., -3.1564e-03,
          -1.5129e-02, -1.8974e-02]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([ 0.0124, -0.0108, -0.0023,  ..., -0.0107,  0.0006,  0.0143],
        device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[ 2.3515e-02,  4.9325e-03, -1.9701e-02,  ..., -5.4721e-03,
          -2.6529e-02, -7.4118e-03],
         [ 1.8091e-02, -1.0

In [39]:
preds = next_mdl(torch.tensor(train_next[:,:-1]).cuda()).detach().cpu().numpy()
d = pd.DataFrame(data=preds)


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_10643/1083022234.py:1 in <cell line: 1>                                    │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_10643/1083022234.py'                 │
│ /home/henry/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1130 in        │
│ _call_impl                                                                                │
│                                                                                           │
│   1127 │   │   # this function, and just call forward.                                    │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1131 │   │   # Do not call functions when jit is used                                   │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /tmp/ipykernel_10643/1898263845.py:11 in forward                                          │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_10643/1898263845.py'                 │
│                                                                                           │
│ /home/henry/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1130 in        │
│ _call_impl                                                                                │
│                                                                                           │
│   1127 │   │   # this function, and just call forward.                                    │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1131 │   │   # Do not call functions when jit is used                                   │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /home/henry/.local/lib/python3.10/site-packages/torch/nn/modules/linear.py:114 in forward │
│                                                                                           │
│   111 │   │   │   init.uniform_(self.bias, -bound, bound)                                 │
│   112 │                                                                                   │
│   113 │   def forward(self, input: Tensor) -> Tensor:                                     │
│ ❱ 114 │   │   return F.linear(input, self.weight, self.bias)                              │
│   115 │                                                                                   │
│   116 │   def extra_repr(self) -> str:                                                    │
│   117 │   │   return 'in_features={}, out_features={}, bias={}'.format(                   │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: CUDA out of memory. Tried to allocate 196.00 MiB (GPU 0; 5.81 GiB total 
capacity; 3.93 GiB already allocated; 84.81 MiB free; 4.61 

In [ ]:
pd.to_csv(d, "next_preds.csv")

In [44]:
def get_conversion(inp):
    out = []
    for i in inp.flatten():
        if i >= 0.4:
            out.append(1)
        else:
            out.append(0)
    return np.array(out)

In [39]:
np.set_printoptions(threshold=np.inf)

In [45]:
print(confusion_matrix(get_conversion(model(train_s[:,:-1]).detach().cpu().numpy()), train_s[:,-1].detach().cpu().numpy().flatten()))
print(confusion_matrix(get_conversion(model(test_s[:,:-1]).detach().cpu().numpy()), test_s[:,-1].detach().cpu().numpy().flatten()))

[[38533   341]
 [  328  5906]]
[[4209  109]
 [ 130  565]]
